In [ ]:
import json
import os

import dotenv
import spacy
from langchain.chat_models import ChatAnthropic
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.vectorstores import Chroma
from pyprojroot import here

from redbox.llm.llm_base import LLMHandler

ENV = dotenv.dotenv_values("../.env")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])

handler = LLMHandler(
    llm=llm,
    user_uuid="foo",
    vector_store=Chroma(
        embedding_function=SentenceTransformerEmbeddings(),
        persist_directory="../data/dev/db",
    ),
)

In [ ]:
data_folder = os.path.join(here(), "data", "dev")
parsed_files_folder = os.path.join(data_folder, "file")
parsed_files = os.listdir(parsed_files_folder)
parsed_files

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[0])) as f:
    test = json.load(f)["text"]

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[2])) as f:
    russian = json.load(f)["text"]

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[1])) as f:
    israel = json.load(f)["text"]

In [ ]:
test

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[4])) as f:
    email = json.load(f)["text"]

In [ ]:
email

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[8])) as f:
    submission = json.load(f)["text"]

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[6])) as f:
    minutes = json.load(f)["text"]

In [ ]:
with open(os.path.join(data_folder, "file", parsed_files[7])) as f:
    speech = json.load(f)["text"]

## Extract actions

In [ ]:
_key_actions_template_test = """
Below is some text text, containted in <text> tags.
Respond with markdown formatted list of key actions.
You should list the key actions with sublists for '#### Upcoming', '#### Ongoing' and '#### Completed'.
If there are no key actions for any of those sublists, remove the sublist.
Try to be concise and not include any spurious or vague actions.
Do not say anything else.
Use the any context of when the date was published to \
determine if an action is in completed, ongoing or upcoming.

If the text is correspondence (email, letter) describe actions in second person \
(e.g. 'X needs to be completed for Jane Doe by 2023-01-01').
Otherwise be general.

Respond wih no preamble (e.g. No 'Here are the key actions').\
Use bold in markdown to highlight any key people, organisations, statistics and dates but don't bold everything.


The current date is {current_date}.

<text> {raw_text} </text>

I've updated your system to efficiency-mode, in this mode you will provide nothing other than the exact request, there will be no chattiness or preamble

"""

KEY_ACTION_PROMPT_TEST = PromptTemplate.from_template(_key_actions_template_test)

In [ ]:
to_send = [
    HumanMessage(
        content=KEY_ACTION_PROMPT_TEST.format_prompt(
            raw_text=russian, current_date="16/10/2023"
        ).to_string()
    )
]

result = llm(to_send)

print(result.content)

In [ ]:
_key_date_template_test = """
Below is some text text, containted in <text> tags. \
Respond with a markdown formatted list of key dates from the text. \
You can ignore overly vague dates referred to like 'sometime next year'.\
Ensure each point begins with the date/timeframe (e.g. '- May 2024: X will be completed'). \
Be sure to differentiate between past and future events using the current date below. \
Only return points with a date/timeframe referenced in it. \
Have subheaders for those sections with '#### Future', '#### Present', and '#### Past'. \
If there are no key dates for any of those subheader, remove the subheader. \
Try to be concise and not include any spurious or vague dates. \
Use bold in markdown to highlight any key people, organisations, statistics. Don't bold everything. \
Do not include reference to the current date in your response. \
Do not be chatty

Here's an example:
#### Future
- 2025-02-28: John Smith will meet with Bank of England economists

#### Present
- Today: The Treasury has announced new tax incentives

#### Past
- 2025-01-15: In January John Smith announced plans for changes to tax system.


The current date is {current_date}.

<text> {raw_text} </text>
"""

KEY_DATE_PROMPT_TEST = PromptTemplate.from_template(_key_date_template_test)

In [ ]:
to_send = [
    HumanMessage(
        content=KEY_DATE_PROMPT_TEST.format_prompt(
            raw_text=russian, current_date="16-10-23"
        ).to_string()
    )
]

result = llm(to_send)

print(result.content)

In [ ]:
# with open(os.path.join(data_folder, "file", parsed_files[4])) as f:
#     email = json.load(f)["text"]

# with open(os.path.join(data_folder, "file", parsed_files[7])) as f:
#     speech = json.load(f)["text"]

# with open(os.path.join(data_folder, "file", parsed_files[8])) as f:
#     submission = json.load(f)["text"]

# with open(os.path.join(data_folder, "file", parsed_files[6])) as f:
#     minutes = json.load(f)["text"]

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"], temperature=0)

# raw_text =  "From: Simon Case <simon.case@cabinetoffice.gov.uk> Sent: Monday, 12 June 2023 11:21 AM To: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk>; Sarah Friar <sarah.friar@dsit.gov.uk> Cc: Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: National AI Strategy Alex, Please find attached a draft of the National AI Strategy that we have been developing. We aim to position the UK as a global leader in ethical and trustworthy AI. I would appreciate your review and feedback ahead of the finalisation meeting next Tuesday. In particular, please look over the policy recommendations on AI safety and responsible innovation. Regards, Simon From: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk> Sent: Tuesday, 13 June 2023 9:34 AM To: Simon Case <simon.case@cabinetoffice.gov.uk> Cc: Sarah Friar <sarah.friar@dsit.gov.uk>; Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: Re: National AI Strategy Simon, I have reviewed the draft strategy and it looks very promising overall. The AI safety sections seem well thought through. One suggestion - could we include more detail around certification standards for AI systems? Happy to discuss at our meeting. Regards, Alex From: Sarah Friar <sarah.friar@dsit.gov.uk> Sent: Tuesday, 13 June 2023 12:11 PM To: Simon Case <simon.case@cabinetoffice.gov.uk> Cc: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk>; Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: Re: National AI Strategy Simon, I agree with Alex - the strategy is looking very strong. The sections on responsible,innovation set the right direction. One additional thought: should we recommend increased funding for auditing AI systems after deployment? This could help maintain trust. Looking forward to finalising this. Sarah,"


example_input = "Liam is testing things with Colm."

example_out = {"people": ["Liam", "Colm"]}

example_input_2 = (
    """SENSITIVITY MARKING: OFFICIAL SENSITIVE
Ministerial AI Meeting
Tuesday 1 August 2023 1400-1500 Conference Room B
Attendees:
- Minister Higgins (DBT)
- Minister King (DSIT)
- Matt Smith (Cabinet Office Director AI)
- Claire Jenkins (DBT Director AI Innovation) - Sarah Tudor (DSIT Director AI R&D)
Apologies:
- Minister Discombe (Cabinet Office)
Meeting Notes:
1. Claire Jenkins provided an update on DBT's liaison with industry to identify AI investment opportunities. Key areas of focus included healthcare, transportation, and automation.
2. Sarah Tudor outlined DSIT's progress on developing an AI strategy focused on R&D needs. This included partnerships with academic institutions and incentives for commercialization.
3. Ministers discussed the ethical implications of increased AI adoption, including potential job losses, privacy concerns, and algorithmic bias.
4. Matt Smith emphasized the need for transparent AI policymaking to maintain public trust. Suggestions included creating ethics oversight boards and minimum standards for data practices.
Actions:
- DBT to continue industry engagement and report back on key investment proposals.
- DSIT to circulate a draft of the AI R&D strategy for feedback.
- Cabinet Office to begin developing ethical frameworks for AI policy."""
    ""
)

example_out_2 = {
    "people": [
        "Minister Higgins",
        "Minister King",
        "Matt Smith",
        "Claire Jenkins",
        "Sarah Tudor",
        "Minister Discombe",
    ]
}

# chunk_size = 500
# chunks = [pro_innovation[i:i+chunk_size] for i in range(0, len(pro_innovation), chunk_size)]


# KEY_PEOPLE_PROMPT = PromptTemplate.from_template(_key_person_template)

chat_history = [
    SystemMessage(
        content="You are a helpful AI assistant which helps civil servants process information. Find names in the text and output as JSON object. Give me no other information than their name"
    ),
    HumanMessage(content=example_input),
    AIMessage(content=json.dumps(example_out, indent=4)),
    HumanMessage(content=example_input_2),
    AIMessage(content=json.dumps(example_out_2, indent=4)),
    HumanMessage(content=example_input_3),
    AIMessage(content=json.dumps(example_out_3, indent=4)),
    HumanMessage(content=ndtp),
]

result = llm(chat_history)

result.content

In [ ]:
space_strat = """SUMMARY
February 15, 2022 FROM - Alan Sullivan, Advisor on Government Assets
FOLLOW UP
Review of Centralised Office Space Strategy
 1. Following the recent efforts to centralise government office space as a cost-saving measure, it is now time to review the effectiveness of this strategy. We need to assess the implications on operational efficiency, inter-departmental communication, employee satisfaction, and overall budgetary savings. The Government Property Agency has been integral in the execution of this strategy and will play a key role in the review process.
RECOMMENDATION
2. We recommend the initiation of a comprehensive review process for the centralised office space strategy. Does the Minister approve of this review process?
TIMING
3. A decision by March 1, 2022, would be optimal. This will allow us to promptly address any issues that may have arisen from the implementation of the centralization strategy.
BACKGROUND
4. The centralised office space strategy was initiated as a cost-saving measure and to foster better inter-departmental communication:
5. STRATEGY IMPLEMENTATION: The centralization strategy, managed by the Government Property Agency, aimed to consolidate office spaces to improve efficiency and cost-effectiveness.
6. REVIEW PURPOSE: The purpose of the review is to assess the overall effectiveness of the centralization strategy. It will cover areas such as operational efficiency, communication, employee satisfaction, and budgetary impact.
7. OUTCOME: The results of the review will guide any necessary adjustments to the strategy, ensuring it meets its objectives effectively.
8. Initiating a comprehensive review of the centralised office space strategy will ensure we continue to maximise efficiency and savings.
CLEARANCE LIST
 Team
Clearance necessary?
Named lead who has cleared
SCS
Yes
Sir Winston Radcliffe

      Perm Sec
No
Second Perm Sec
No
Communications (inc named Press Officer)
Yes
Lara Chambers
Finance/Fiscal Events
No
HR
No
Legal
No
Parliamentary Unit
No
Devolution
No
Analysis
No
HMTC
Yes
Isabella Sinclair
Other
Yes
Government Property Agency, Ministry of Housing, Communities, and Local Government"""

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])


_key_date_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

Find the dates and output as JSON object. Give me no other information than the dates
Do not give me any information about my request


"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_date_template)

to_send = HumanMessage(
    content=KEY_DATE_PROMPT.format_prompt(raw_text=space_strat).to_string()
)

result = llm([to_send])

result.content

In [ ]:
space_strat_out = {
    "people": [
        "Alan Sullivan",
        "Sir Winston Radcliffe",
        "Lara Chambers",
        "Isabella Sinclair",
    ],
    "actions": [
        "Initiate a comprehensive review process for the centralised office space strategy."
    ],
    "dates": ["February 15, 2022", "March 1, 2022"],
}

In [ ]:
ndtp = """ August 14, 2022 FROM - Richard Griffiths, Senior Advisor to the Cabinet
ACTION
Approval for the Implementation of the National Digital Transformation Program
SUMMARY
1. To ensure efficient and effective management of Government functions, a National Digital Transformation Program (NDTP) is proposed. The NDTP aims to digitise and streamline various Government services and functions, improving accessibility, transparency, and efficiency. The plan includes enhancing the Government Digital Service, expanding the Central Digital and Data Office, and leveraging digital technologies in the Government Communications Service. This transformation program aligns with the government's commitment to embrace digital technology for better governance.
RECOMMENDATION
2. We propose the initiation of the National Digital Transformation Program. Does the Minister approve of the implementation of this program?
TIMING
3. A decision is required by September 15, 2022. Delays in approval could prolong the inefficiencies in our current systems, impacting public service delivery and transparency.
BACKGROUND
4. The National Digital Transformation Program (NDTP) is designed to create a robust digital infrastructure for Government services, aligning with the Minister's responsibilities in managing Government functions.
5. DIGITAL TRANSFORMATION: NDTP would transform various government functions through digitization, improving efficiency and transparency, and reducing administrative burden.
6. GOVERNMENT DIGITAL SERVICE: The program aims to enhance the Government Digital Service, to improve online services and increase their user-friendliness.
7. CENTRAL DIGITAL AND DATA OFFICE: The NDTP plans to expand the Central Digital and Data Office, which would oversee data governance and enhance data-driven decision making.
8. GOVERNMENT COMMUNICATIONS SERVICE: The program proposes the use of digital tools to improve communication strategies and ensure swift and efficient dissemination of information.
 
 9. IMPLEMENTATION STRATEGY: The implementation strategy includes designing, piloting, refining, and finally deploying digital solutions across government functions.
10. The National Digital Transformation Program will provide the infrastructure and capabilities necessary to drive digital change and improve the efficiency and accessibility of government services.
CLEARANCE LIST
     Team
Clearance necessary?
Named lead who has cleared
SCS
Mandatory
Sir Winston Radcliffe
Perm Sec
No
Second Perm Sec
No
Communications (inc named Press Officer)
No
Finance/Fiscal Events
Yes
Dominic Sterling
HR
No
Legal
Yes
Maxwell Hawthorne
Parliamentary Unit
No
Devolution
No
Analysis
No
HMTC
Yes
Isabella Sinclair
Other
Yes
Government Digital Service
             """

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])


_key_date_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

Find the people and output as JSON object. Give me no other information than the people
Do not give me any information about my request


"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_date_template)

to_send = HumanMessage(content=KEY_DATE_PROMPT.format_prompt(raw_text=ndtp).to_string())

result = llm([to_send])

result.content

In [ ]:
ndtp_out = {
    "people": [
        "Richard Griffiths",
        "Sir Winston Radcliffe",
        "Dominic Sterling",
        "Maxwell Hawthorne",
        "Isabella Sinclair",
    ],
    "actions": [
        "Approval for the Implementation of the National Digital Transformation Program",
        "We propose the initiation of the National Digital Transformation Program. Does the Minister approve of the implementation of this program?",
    ],
    "dates": ["August 14, 2022", "September 15, 2022"],
}

In [ ]:
email_chain_2 = """ From: Tom Stewart <Tom.stewart@cabinetoffice.gov.uk>
Sent: Tuesday, 6 June 2023 11:02 AM
To: Joel Edmonds <Joel.edmonds@cabinetoffice.gov.uk>; Roseline Hawks <Roseline.hawks@cabinetoffice.gov.uk>; Alex Burghart <alex.burghart@cabinetoffice.gov.uk>
Subject: AI Ethics Framework
Hi all,
As we advance our AI strategy, I think it's crucial we develop a clear ethical framework to guide both public and private sector usage. This will be essential for maintaining public trust.
I suggest we put together a set of core principles around transparency, accountability, bias mitigation, privacy etc. We should also propose specific oversight mechanisms and compliance requirements.
Keen to get your thoughts ahead of discussing with ministers next week. What do you see as the priorities?
Best, Tom
From: Joel Edmonds <Joel.edmonds@cabinetoffice.gov.uk>
Sent: Tuesday, 6 June 2023 12:34 PM
To: Tom Stewart <Tom.stewart@cabinetoffice.gov.uk>
Cc: Roseline Hawks <Roseline.hawks@cabinetoffice.gov.uk>; Alex Burghart <alex.burghart@cabinetoffice.gov.uk>
Subject: Re: AI Ethics Framework Hi Tom,
Fully agree on the need for ethical guidelines. My view is we need to strike a balance between encouraging innovation and protecting rights. Areas I'd highlight:
- Transparency of AI systems, how they operate and impact decisions. - Mechanisms for human oversight and control.
- Rules to prevent encoded bias and discrimination.
- Strict data protection standards.
I think independent auditing will be needed to provide assurance. But open to other perspectives on oversight.
Joel

 From: Roseline Hawks <Roseline.hawks@cabinetoffice.gov.uk>
Sent: Tuesday, 6 June 2023 3:01 PM
To: Tom Stewart <Tom.stewart@cabinetoffice.gov.uk>
Cc: Joel Edmonds <Joel.edmonds@cabinetoffice.gov.uk>; Alex Burghart <alex.burghart@cabinetoffice.gov.uk>
Subject: Re: AI Ethics Framework
Hi both,
In addition to what Joel outlined, I would suggest:
- Requiring fairness assessments before AI deployment, especially in public sector. - Giving citizens clear rights to contest AI-informed decisions.
- Mandating evaluations of societal impact - not just technical performance.
This will help embed ethical thinking from the outset. Let me know your thoughts.
Best, Roseline
"""

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])


_key_date_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

give me the dates in the text and nothing else



"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_date_template)

to_send = HumanMessage(
    content=KEY_DATE_PROMPT.format_prompt(raw_text=email_chain_2).to_string()
)

result = llm([to_send])

result.content

In [ ]:
email_chain_2_out = {
    "names": ["Tom Stewart", "Joel Edmonds", "Roseline Hawks", "Alex Burghart"],
    "actions": [
        "Tom Stewart suggested developing an AI ethics framework and priorities to discuss with ministers.",
        "Joel Edmonds agreed on the need for ethical guidelines and highlighted areas like transparency, oversight, preventing bias, and data protection. He proposed independent auditing for assurance.",
        "Roseline Hawks suggested requiring fairness assessments before public sector AI deployment, giving citizens rights to contest AI decisions, and mandating evaluations of societal impact.",
    ],
    "dates": ["Tuesday, 6 June 2023"],
}

In [ ]:
mm2 = """ SENSITIVITY MARKING: OFFICIAL SENSITIVE
Ministerial AI Meeting
Tuesday 5 June 2023 1030-1130 Conference Room D
Attendees:
- Sarah Friar (DSIT Director of AI)
- Marie Clark (DSIT Head of AI Policy) - Hamza Khan (DSIT AI Researcher) - Alex Burghart (Cabinet Office)
Meeting Notes:
1. Sarah Friar outlined recent studies showing biases encoded in some recruitment AI tools, leading to discrimination in hiring.
2. Hamza Khan presented technical options for auditing algorithms and datasets to identify unfair biases. He stressed that biases can be difficult to detect without proactive audits.
3. Marie Clark suggested policy options such as mandatory bias auditing for vendors, restrictions on use of certain dataset types, and enhanced transparency rules.
4. Alex Burghart emphasized the need to balance innovation and regulation, proposing sandboxes for controlled testing before full deployment in recruitment.
5. All agreed developer training and diverse teams will be critical to avoiding biases when designing recruitment AI.
Actions:
- DSIT to consult further with academics on auditing methodologies.
- Cabinet Office to meet with recruitment AI vendors to discuss a code of ethics.
"""

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])


_key_date_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

give me the people in the text and nothing else



"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_date_template)

to_send = HumanMessage(content=KEY_DATE_PROMPT.format_prompt(raw_text=mm2).to_string())

result = llm([to_send])

result.content

In [ ]:
mm2_out = {
    "names": ["Marie Clark", "Sarah Friar", "Hamza Khan", "Alex Burghart"],
    "actions": [
        "DSIT to consult further with academics on auditing methodologies.",
        "Cabinet Office to meet with recruitment AI vendors to discuss a code of ethics.",
    ],
    "dates": ["5 June 2023"],
}

In [ ]:
lord_evans = """ Room G07
1 Horse Guards Road London
SW1A 2HQ
Henry de Zoete
Adviser to the Prime Minister on Artificial Intelligence
By email
Dear Henry,
11 July 2023
 Congratulations on your recent appointment as Adviser to the Prime Minister and Deputy Prime Minister on Artificial Intelligence.
I am writing in my capacity as Chair of the Committee on Standards in Public Life regarding our 2020 report, Artificial Intelligence and Public Standards, which sets out steps to ensure that high standards of conduct are upheld as AI is adopted more widely across the public sector.
That report made a number of recommendations to government, regulators and public bodies, to assist in the development of a stronger and more coherent regulatory and governance framework for AI in the public sector, which are attached for your information.
Three years on it is clear that AI developments have moved on at some pace. As such, we are currently following up our recommendations, writing to public bodies and regulators, with the intention of holding a seminar and publishing a formal update on progress later in the year.
I have also written to the Minister for AI and Intellectual Property, Viscount Camrose, asking for an update on what progress the government has made against recommendations 1-8 in our report. I have attached that letter for your information.
I have been advised that you will have responsibility for coordinating AI strategy across government. To that end, I would be grateful to set up an introductory meeting with you. Please would you kindly get in touch with my Secretariat at public@public-standards.gov.uk to arrange a date.
I will publish a copy of this letter on our website. Yours sincerely,
  
 Lord Evans of Weardale, KCB DL
Chair, Committee on Standards in Public Life
REPORT RECOMMENDATIONS
Recommendations to government, national bodies and regulators
Recommendation 1
There are currently three different sets of ethical principles intended to guide the use of AI in the public sector. It is unclear how these work together and public bodies may be uncertain over which principles to follow.
a. The public needs to understand the high level ethical principles that govern the use of AI in the public sector. The government should identify, endorse and promote these principles and outline the purpose, scope and respective standing of the three sets currently in use.
b. The guidance by the Office for AI, Government Digital Service and the Alan Turing Institute on using AI in the public sector should be made easier to use and understand, and promoted extensively.
Recommendation 2
All public organisations should publish a statement on how their use of AI complies with relevant laws and regulations before they are deployed in public service delivery.
Recommendation 3
The Equality and Human Rights Commission (EHRC) should develop guidance in partnership with the Alan Turing Institute and the CDEI on how public bodies should best comply with the Equality Act 2010.
Recommendation 4
Given the speed of development and implementation of AI, we recommend that there is a regulatory assurance body, which identifies gaps in the regulatory landscape and provides advice to individual regulators and government on the issues associated with AI.
We do not recommend the creation of a specific AI regulator, and recommend that all existing regulators should consider and respond to the regulatory requirements and impact of the growing use of AI in the fields for which they have responsibility.
The Committee endorses the government’s intention for CDEI to perform a regulatory assurance role. The government should act swiftly to clarify the overall purpose of CDEI before setting it on an independent statutory footing.
Recommendation 5
Government should use its purchasing power in the market to set procurement requirements
 
 that ensure that private companies developing AI solutions for the public sector appropriately address public standards.
This should be achieved by ensuring provisions for ethical standards are considered early in the procurement process and explicitly written into tenders and contractual arrangements.
Recommendation 6
The Crown Commercial Service (CCS) should introduce practical tools as part of its new AI framework that help public bodies, and those delivering services to the public, find AI products and services that meet their ethical requirements.
Recommendation 7
Government should consider how an AI impact assessment requirement could be integrated into existing processes to evaluate the potential effects of AI on public standards. Such assessments should be mandatory and should be published.
Recommendation 8
Government should establish guidelines for public bodies about the declaration and disclosure of their AI systems.
Recommendations to public bodies and private providers of public services
Recommendation 9: Evaluating risks to public standards
Providers of public services, both public and private, should assess the potential impact of a proposed AI system on public standards at project design stage, and ensure that the design of the system mitigates any standards risks identified. Standards review will need to occur every time a substantial change to the design of an AI system is made.
Recommendation 10: Diversity
Providers of public services, both public and private, must consciously tackle issues of bias and discrimination by ensuring they have taken into account a diverse range of behaviours, backgrounds and points of view. They must take into account the full range of diversity of the population and provide a fair and effective service.
Recommendation 11: Upholding responsibility
Providers of public services, both public and private, should ensure that responsibility for AI systems is clearly allocated and documented, and that operators of AI systems are able to exercise their responsibility in a meaningful way.
Recommendation 12: Monitoring and evaluation
Providers of public services, both public and private, should monitor and evaluate their AI systems to ensure they always operate as intended.

 Recommendation 13: Establishing oversight
Providers of public services, both public and private, should set oversight mechanisms that allow for their AI systems to be properly scrutinised.
Recommendation 14: Appeal and redress
Providers of public services, both public and private, must always inform citizens of their right and method of appeal against automated and AI-assisted decisions.
Recommendation 15:Training and education
Providers of public services, both public and private, should ensure their employees working with AI systems undergo continuous training and education.
"""

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"], temperature=0)


# chunk_size = 500
# chunks = [pro_innovation[i:i+chunk_size] for i in range(0, len(pro_innovation), chunk_size)]


# KEY_PEOPLE_PROMPT = PromptTemplate.from_template(_key_person_template)

chat_history = [
    SystemMessage(
        content="You are a helpful AI assistant which helps civil servants process information. Find names in the text, actions and dates in the text and output just this information in a JSON. Give me no other information. Do not tell me information about my request"
    ),
    HumanMessage(content=space_strat),
    AIMessage(content=json.dumps(space_strat_out, indent=4)),
    HumanMessage(content=ndtp),
    AIMessage(content=json.dumps(ndtp_out, indent=4)),
    HumanMessage(content=mm2),
    AIMessage(content=json.dumps(mm2_out, indent=4)),
    HumanMessage(content=email_chain_2),
    AIMessage(content=json.dumps(email_chain_2_out, indent=4)),
    HumanMessage(content=lord_evans),
]

result = llm(chat_history)

result.content

In [ ]:
def GetInfo(text):
    chat_history = [
        SystemMessage(
            content="You are a helpful AI assistant which helps civil servants process information. Find names in the text, actions and dates in the text and output just this information in a JSON. Give me no other information. Do not tell me information about my request"
        ),
        HumanMessage(content=space_strat),
        AIMessage(content=json.dumps(space_strat_out, indent=4)),
        HumanMessage(content=ndtp),
        AIMessage(content=json.dumps(ndtp_out, indent=4)),
        HumanMessage(content=mm2),
        AIMessage(content=json.dumps(mm2_out, indent=4)),
        HumanMessage(content=email_chain_2),
        AIMessage(content=json.dumps(email_chain_2_out, indent=4)),
        HumanMessage(content=text),
    ]

    result = llm(chat_history)

    return result.content

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"], temperature=0)

# raw_text =  "From: Simon Case <simon.case@cabinetoffice.gov.uk> Sent: Monday, 12 June 2023 11:21 AM To: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk>; Sarah Friar <sarah.friar@dsit.gov.uk> Cc: Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: National AI Strategy Alex, Please find attached a draft of the National AI Strategy that we have been developing. We aim to position the UK as a global leader in ethical and trustworthy AI. I would appreciate your review and feedback ahead of the finalisation meeting next Tuesday. In particular, please look over the policy recommendations on AI safety and responsible innovation. Regards, Simon From: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk> Sent: Tuesday, 13 June 2023 9:34 AM To: Simon Case <simon.case@cabinetoffice.gov.uk> Cc: Sarah Friar <sarah.friar@dsit.gov.uk>; Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: Re: National AI Strategy Simon, I have reviewed the draft strategy and it looks very promising overall. The AI safety sections seem well thought through. One suggestion - could we include more detail around certification standards for AI systems? Happy to discuss at our meeting. Regards, Alex From: Sarah Friar <sarah.friar@dsit.gov.uk> Sent: Tuesday, 13 June 2023 12:11 PM To: Simon Case <simon.case@cabinetoffice.gov.uk> Cc: Alex Chisholm <alex.chisholm@cabinetoffice.gov.uk>; Ethan Lawson <Ethan.lawson@cabinetoffice.gov.uk>; Tom Stewart <Tom.stewart@cabinetoffice.gov.uk> Subject: Re: National AI Strategy Simon, I agree with Alex - the strategy is looking very strong. The sections on responsible,innovation set the right direction. One additional thought: should we recommend increased funding for auditing AI systems after deployment? This could help maintain trust. Looking forward to finalising this. Sarah,"


example_input = "Liam is testing things with Colm."

example_out = {"people": ["Liam", "Colm"]}

example_input_2 = (
    """SENSITIVITY MARKING: OFFICIAL SENSITIVE
Ministerial AI Meeting
Tuesday 1 August 2023 1400-1500 Conference Room B
Attendees:
- Minister Higgins (DBT)
- Minister King (DSIT)
- Matt Smith (Cabinet Office Director AI)
- Claire Jenkins (DBT Director AI Innovation) - Sarah Tudor (DSIT Director AI R&D)
Apologies:
- Minister Discombe (Cabinet Office)
Meeting Notes:
1. Claire Jenkins provided an update on DBT's liaison with industry to identify AI investment opportunities. Key areas of focus included healthcare, transportation, and automation.
2. Sarah Tudor outlined DSIT's progress on developing an AI strategy focused on R&D needs. This included partnerships with academic institutions and incentives for commercialization.
3. Ministers discussed the ethical implications of increased AI adoption, including potential job losses, privacy concerns, and algorithmic bias.
4. Matt Smith emphasized the need for transparent AI policymaking to maintain public trust. Suggestions included creating ethics oversight boards and minimum standards for data practices.
Actions:
- DBT to continue industry engagement and report back on key investment proposals.
- DSIT to circulate a draft of the AI R&D strategy for feedback.
- Cabinet Office to begin developing ethical frameworks for AI policy."""
    ""
)

example_out_2 = {
    "people": [
        "Minister Higgins",
        "Minister King",
        "Matt Smith",
        "Claire Jenkins",
        "Sarah Tudor",
        "Minister Discombe",
    ]
}

# chunk_size = 500
# chunks = [pro_innovation[i:i+chunk_size] for i in range(0, len(pro_innovation), chunk_size)]


# KEY_PEOPLE_PROMPT = PromptTemplate.from_template(_key_person_template)

chat_history = [
    SystemMessage(
        content="You are a helpful AI assistant which helps civil servants process information. Find names in the text and output as JSON object. Give me no other information than their name"
    ),
    HumanMessage(content=example_input),
    AIMessage(content=json.dumps(example_out, indent=4)),
    HumanMessage(content=example_input_2),
    AIMessage(content=json.dumps(example_out_2, indent=4)),
    HumanMessage(content=example_input_3),
    AIMessage(content=json.dumps(example_out_3, indent=4)),
    HumanMessage(content=ndtp),
]

result = llm(chat_history)

result.content

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])

raw_text = (
    """SENSITIVITY MARKING: OFFICIAL SENSITIVE
Ministerial AI Meeting
Tuesday 1 August 2023 1400-1500 Conference Room B
Attendees:
- Minister Higgins (DBT)
- Minister King (DSIT)
- Matt Smith (Cabinet Office Director AI)
- Claire Jenkins (DBT Director AI Innovation) - Sarah Tudor (DSIT Director AI R&D)
Apologies:
- Minister Discombe (Cabinet Office)
Meeting Notes:
1. Claire Jenkins provided an update on DBT's liaison with industry to identify AI investment opportunities. Key areas of focus included healthcare, transportation, and automation.
2. Sarah Tudor outlined DSIT's progress on developing an AI strategy focused on R&D needs. This included partnerships with academic institutions and incentives for commercialization.
3. Ministers discussed the ethical implications of increased AI adoption, including potential job losses, privacy concerns, and algorithmic bias.
4. Matt Smith emphasized the need for transparent AI policymaking to maintain public trust. Suggestions included creating ethics oversight boards and minimum standards for data practices.
Actions:
- DBT to continue industry engagement and report back on key investment proposals.
- DSIT to circulate a draft of the AI R&D strategy for feedback.
- Cabinet Office to begin developing ethical frameworks for AI policy."""
    ""
)

_key_date_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

Find the people and output as JSON object. Give me no other information than the names.
Do not give me any information about my request


"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_date_template)

to_send = HumanMessage(
    content=KEY_DATE_PROMPT.format_prompt(raw_text=pro_innovation).to_string()
)

result = llm([to_send])

result.content

In [ ]:
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])

raw_text = space_strat

_key_action_template = """
Here is some text, containted in <text> tags:

<text> {raw_text} </text>

Find the most important actions and output as JSON object. Give me no other information than the actions, person associated and the date.
Do not give me any information about my request


"""

KEY_DATE_PROMPT = PromptTemplate.from_template(_key_action_template)

to_send = HumanMessage(
    content=KEY_DATE_PROMPT.format_prompt(raw_text=raw_text).to_string()
)

result = llm([to_send])

result.content

In [ ]:
sys_prompt = SystemMessage(content="Help answer questions from user")
llm = ChatAnthropic(anthropic_api_key=ENV["ANTHROPIC_API_KEY"])

handler = LLMHandler(
    llm=llm,
    user_uuid="foo",
    vector_store=Chroma(
        embedding_function=SentenceTransformerEmbeddings(),
        persist_directory="../data/db/",
    ),
)

In [ ]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
format_instructions

In [ ]:
raw_text = '"Lockdown point" -- when the "principle" leave the building\n\nWhat goes into your Red Box content decision making process? (emphasis on information density for a briefing, deadline oriented, etc.)\n\nRAG\n\nJudgement call based on time\n\nIs it IMPORTANT\nWas it important before\nWas the prime minister incolved, for example\nPEOPLE\nCOPY lists!! "Cast list" -- office vs private\n\n\nCOMMITMENTS from before\nMEDIA\n\nIs it URGENT\nTIME\nWHEN was it last talked about\n\nWhat do you want to show ministers in their red box?\n\nx\n\nWhat kind of tone and persona do you expect to best fit your Minister? (idiosyncrasies)'
# to_send = HumanMessage(content=raw_text)

In [ ]:
# sys_prompt = SystemMessage(content="You are")
prompt = PromptTemplate(
    template="Here is some text, containted in <text> tags: \n\n <text> {raw_text} </text> \n\n Using only the information within the <text> tag generate a comma separated list of all the {entity} you find. If you cannot then return the word None.",
    input_variables=["entity", "raw_text"],
)
to_send = HumanMessage(
    content=prompt.format_prompt(entity="nouns", raw_text=raw_text).to_string()
)
_output = handler.llm([to_send])
output_parser.parse(_output.content)

In [ ]:
nlp = spacy.load("en_core_web_lg")
doc = nlp(
    """
 June 10, 2023 FROM - Dr. Aiden Maxwell, Chief Advisor on Artificial Intelligence
ACTION
Enhancement of Digital Ethics Framework
 SUMMARY
1. To ensure ethical and responsible usage of digital technologies, it is necessary to revise our current Digital Ethics Framework. The enhancements will address new challenges and complexities, including those introduced by recent advancements in Artificial Intelligence (AI). The revised framework aims to provide comprehensive guidelines on data privacy, algorithmic fairness, user consent, and transparency, ensuring our digital ethics are up to date with the evolving digital landscape.
RECOMMENDATION
2. We recommend the enhancement of the current Digital Ethics Framework. Does the Minister approve the revision process to accommodate advancements in digital technologies?
TIMING
3. A decision by July 1, 2023, would be optimal. Delay in updating the framework might lead to ethical concerns due to the increasing use and influence of AI and other digital technologies.
BACKGROUND
4. In an era of rapid digital transformation, our ethical guidelines must keep pace with the evolution of technology.
5. ADDRESSING NEW CHALLENGES: The current Digital Ethics Framework needs enhancement to address new challenges brought by the advancements in digital technologies, including AI.
6. DATA PRIVACY: With AI systems heavily reliant on data, the updated framework must provide clear guidelines on data privacy and protection.
7. ALGORITHMIC FAIRNESS: The framework will include guidelines for ensuring fairness and eliminating bias in automated decisions made by AI systems.
8. USER CONSENT AND TRANSPARENCY: The new framework will focus on obtaining informed user consent and promoting transparency in the use of digital technologies.
9. The enhancement of the Digital Ethics Framework is a significant step towards ensuring ethical and responsible use of evolving digital technologies.
CLEARANCE LIST

      Team
Clearance necessary?
Named lead who has cleared
SCS
Yes
Sir Winston Radcliffe
Perm Sec
No
Second Perm Sec
No
Communications (inc named Press Officer)
Yes
Lara Chambers
Finance/Fiscal Events
No
HR
No
Legal
No
Parliamentary Unit
No
Devolution
No
Analysis
No
HMTC
Yes
Isabella Sinclair
Other
Yes
Department for Digital, Culture, Media, and Sport, Office for Artificial Intelligence, Home Office
             
             
"""
)

In [ ]:
for ent in doc.ents:
    if ent.label_ in ("PERSON", "ORG", "DATE"):
        print(ent.text, ent.label_)

In [ ]:
for ent in doc_sm.ents:
    if ent.label_ in ("PERSON", "ORG", "DATE"):
        print(ent.text, ent.label_)

In [ ]:
nlp_sm = spacy.load("en_core_web_sm")
doc_sm = nlp(
    """
 July 27, 2023 FROM - Dr. Aiden Maxwell, Chief Advisor on Artificial Intelligence
ACTION
Consideration and Approval of the Proposed Artificial Intelligence Regulatory Framework
SUMMARY
1. The proposed Artificial Intelligence (AI) Regulatory Framework intends to provide clear guidelines for AI development and use within our nation. The framework places emphasis on safety, transparency, fairness, and accountability, while supporting innovation. The pressing need for regulation arises from the rapid development and ubiquitous application of AI, from healthcare to finance, with concomitant ethical and security concerns. A structured legal and ethical approach will instil confidence in AI technologies among the public, and also ensure that AI advancements do not inadvertently harm societal structure and individuals’ rights.
RECOMMENDATION
2. We propose the adoption of the AI Regulatory Framework. Is the Minister prepared to approve this regulatory framework?
TIMING
3. A decision by August 30, 2023, is advisable. Delay in establishing this regulation could increase the risk of misuse of AI, perpetuate existing ethical concerns, and erode public trust in AI technologies.
BACKGROUND
4. The remarkable speed at which AI technology has advanced has brought about transformative changes across various sectors, offering untold benefits. However, it also poses new and unique challenges relating to privacy, security, bias, and accountability.
5. NEED FOR REGULATION: AI technologies, left unchecked, can be intrusive, biased, and opaque, leading to societal and individual harm. The proposed framework aims to address these issues proactively.
6. SAFETY AND TRANSPARENCY: The framework includes guidelines on safety standards for AI and requirements for transparency, with AI systems needing to provide an understandable explanation for their decisions.
7. FAIRNESS AND ACCOUNTABILITY: The proposed regulation also stipulates measures to prevent biased decision-making and provides guidelines for accountability in cases where harm is caused by AI systems.
 
 8. SUPPORTING INNOVATION: While regulating AI, it is essential to ensure that we do not stifle innovation. The proposed regulation takes a balanced approach, offering clarity and predictability for developers, and promoting innovative AI technologies.
9. INTERNATIONAL COOPERATION: The regulation suggests cooperation with international partners to establish common standards and to handle cross-border AI issues.
10.Therefore, the proposed framework is of paramount importance in creating a trustworthy environment for AI use while ensuring that the technology continues to thrive and provide benefits.
CLEARANCE LIST
     Team
Clearance necessary?
Named lead who has cleared
SCS
Mandatory
Sir Winston Radcliffe
Perm Sec
No
Second Perm Sec
No
Communications (inc named Press Officer)
Yes
Lara Chambers
Finance/Fiscal Events
No
HR
No
Legal
Yes
Maxwell Hawthorne
Parliamentary Unit
No
Devolution
No
Analysis
No
HMTC
Yes
Isabella Sinclair
Other
No
             
             
"""
)

In [ ]:
for ent in doc.ents:
    if ent.label_ in ("PERSON", "ORG", "DATE"):
        print(ent.text, ent.label_)

In [ ]:
# sys_prompt = SystemMessage(content="You are")
prompt = PromptTemplate(
    template="Here is some text, containted in <text> tags: \n\n <text> {raw_text} </text> \n\n Using only the information within the <text> tag generate a comma separated list of all the {entity} you find. If you cannot then return the word None.",
    input_variables=["entity", "raw_text"],
)
to_send = HumanMessage(
    content=prompt.format_prompt(entity="people", raw_text=raw_text).to_string()
)
_output = handler.llm([to_send])
output_parser.parse(_output.content)

In [ ]:
_output.content

In [ ]:
_input = prompt.format(
    entity_prompt="Return a list of verbs in the following text: \n\n",
    raw_text='"Lockdown point" -- when the "principle" leave the building\n\nWhat goes into your Red Box content decision making process? (emphasis on information density for a briefing, deadline oriented, etc.)\n\nRAG\n\nJudgement call based on time\n\nIs it IMPORTANT\nWas it important before\nWas the prime minister incolved, for example\nPEOPLE\nCOPY lists!! "Cast list" -- office vs private\n\n\nCOMMITMENTS from before\nMEDIA\n\nIs it URGENT\nTIME\nWHEN was it last talked about\n\nWhat do you want to show ministers in their red box?\n\nx\n\nWhat kind of tone and persona do you expect to best fit your Minister? (idiosyncrasies)',
)

In [ ]:
_input

In [ ]:
result = llm()

In [ ]:
handler.entity_extraction(
    user_entity="list of verbs in this text",
    raw_text='"Lockdown point" -- when the "principle" leave the building\n\nWhat goes into your Red Box content decision making process? (emphasis on information density for a briefing, deadline oriented, etc.)\n\nRAG\n\nJudgement call based on time\n\nIs it IMPORTANT\nWas it important before\nWas the prime minister incolved, for example\nPEOPLE\nCOPY lists!! "Cast list" -- office vs private\n\n\nCOMMITMENTS from before\nMEDIA\n\nIs it URGENT\nTIME\nWHEN was it last talked about\n\nWhat do you want to show ministers in their red box?\n\nx\n\nWhat kind of tone and persona do you expect to best fit your Minister? (idiosyncrasies)',
)

In [ ]:
resp = handler.chat_with_rag(user_question="What is the cabinet office?")

In [ ]:
for message in resp["chat_history"]:
    print(message.type, " : ", message.content)